In [1]:
#-*-coding:utf-8-*- #

# import library
import logging
import math
import random
import mxnet as mx
import numpy as np

In [2]:
# 打开调试信息的显示
logging.getLogger().setLevel(logging.DEBUG) 

In [3]:
n_sample = 1000 # sample data number
batch_size = 1 # batch size
learning_rate = 0.1 # learning rate
n_epoch = 1 # epoch number

In [4]:
# 每个数据点是在 (0,1) 之间的 2 个随机数
train_in = [[ random.uniform(0, 1) for c in range(2)] for n in range(n_sample)] 

# 期望输出，先初始化为 0
train_out = [0 for n in range(n_sample)]

In [5]:
for i in range(n_sample):
    # 每个数据点的期望输出是 2 个输入数中的大者
    train_out[i] = max(train_in[i][0], train_in[i][1])

In [6]:
train_iter = mx.io.NDArrayIter(data = np.array(train_in), label = {'reg_label':np.array(train_out)}, batch_size = batch_size, shuffle = True)

src = mx.sym.Variable('data')

fc  = mx.sym.FullyConnected(data = src, num_hidden = 1, name = 'fc')

net = mx.sym.LinearRegressionOutput(data = fc, name = 'reg')

module = mx.mod.Module(symbol = net, label_names = (['reg_label']))

In [7]:
def epoch_callback(epoch, symbol, arg_params, aux_params):
    for k in arg_params: # 对于所有参数…
        print(k) # 输出参数名
        print(arg_params[k].asnumpy()) # 参数值，转为 NumPy 数组，输出更美观
        
module.fit(
    train_iter, # 训练数据的迭代器
    eval_data = None, # 在此只训练，不使用测试数据
    eval_metric = mx.metric.create('mse'), # 输出 MSE 损失信息
    optimizer = 'sgd', # 梯度下降算法为 SGD
    # 设置学习速率
    optimizer_params = {'learning_rate': learning_rate}, 
    num_epoch = n_epoch, # 训练 epoch 数
    # 每经过 100 个 batch 输出训练速度 
    batch_end_callback = mx.callback.Speedometer(batch_size, 100), 
    epoch_end_callback = epoch_callback, # 完成每个 epoch 后调用 epoch_callback
)

INFO:root:Epoch[0] Batch [100]	Speed: 2614.25 samples/sec	mse=0.043387
INFO:root:Epoch[0] Batch [200]	Speed: 2357.82 samples/sec	mse=0.018802
INFO:root:Epoch[0] Batch [300]	Speed: 2536.53 samples/sec	mse=0.018297
INFO:root:Epoch[0] Batch [400]	Speed: 2267.73 samples/sec	mse=0.014475
INFO:root:Epoch[0] Batch [500]	Speed: 2318.90 samples/sec	mse=0.015596
INFO:root:Epoch[0] Batch [600]	Speed: 2244.83 samples/sec	mse=0.016810
INFO:root:Epoch[0] Batch [700]	Speed: 2448.81 samples/sec	mse=0.014367
INFO:root:Epoch[0] Batch [800]	Speed: 2367.92 samples/sec	mse=0.012448
INFO:root:Epoch[0] Batch [900]	Speed: 2426.36 samples/sec	mse=0.011891
INFO:root:Epoch[0] Train-mse=0.017026
INFO:root:Epoch[0] Time cost=0.704


fc_weight
[[ 0.48085973  0.48560432]]
fc_bias
[ 0.17381699]
